In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
box = pd.read_csv(r'C:\Users\Admin\Desktop\Datathon\Problem 2\boxes.csv')
purchase= pd.read_csv(r'C:\Users\Admin\Desktop\Datathon\Problem 2\purchase.csv')

In [3]:
box.head()

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96
2,3,Premium,Home Delivery - CoD,0.0,2.4,11.96
3,4,Premium,Home Delivery - CoD,0.0,2.5,11.96
4,5,Premium,Home Delivery - CoD,0.0,2.6,11.96


In [4]:
purchase.head()

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT
0,1/2/2019,2CED678A247,12.0,1.0
1,1/2/2019,2BF58D91BA1,12.0,1.0
2,1/2/2019,2C15B86534E,99.0,1.0
3,1/2/2019,2C32D9A859A,6.0,1.0
4,1/2/2019,2C7A55404D1,4.0,1.0


In [5]:
purchase.dropna(subset=['BOX_ID', 'BOX_COUNT'], inplace=True)

In [6]:
# Convert PURCHASE_DATE to datetime
purchase['PURCHASE_DATE'] = pd.to_datetime(purchase['PURCHASE_DATE'], format='%d/%m/%Y')

# Convert BOX_ID to integer
purchase['BOX_ID'] = purchase['BOX_ID'].astype('int32')

# Convert BOX_COUNT to integer
purchase['BOX_COUNT'] = purchase['BOX_COUNT'].astype('int32')

In [7]:
box['BOX_ID'] = box['BOX_ID'].astype('int32')

In [8]:
box = box.drop_duplicates()
purchase = purchase.drop_duplicates()

In [9]:
df = pd.merge(box, purchase, how='inner')
df.head()

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96,2019-02-04,2C88D36D1FC,1
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2019-01-02,2BF011BDB38,1
2,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2019-01-02,2CA0EE8F2B3,1
3,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2019-01-02,2C623730B09,1
4,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2019-01-02,2CA6CE1054F,1


Creating aggregrated User Profiles

In [10]:
user_profiles = df.groupby('MAGIC_KEY').agg({
    'BOX_ID': lambda x: x.value_counts().idxmax(),  # Most frequently purchased box type
    'MILK': 'mean',                                # Average quantity of milk
    'MEAT': 'mean',                                # Average quantity of meat
    'DELIVERY_OPTION': lambda x: x.mode()[0],      # Most common delivery option
    'QUALITY': lambda x: x.mode()[0],              # Most common quality preferred
    'UNIT_PRICE': 'mean',                          # Average unit price paid
    'BOX_COUNT': 'sum'                             # Total number of boxes purchased
}).rename(columns={
    'BOX_ID': 'MOST_FREQUENT_BOX_ID',
    'MILK': 'AVG_MILK',
    'MEAT': 'AVG_MEAT',
    'DELIVERY_OPTION': 'PREFERRED_DELIVERY_OPTION',
    'QUALITY': 'PREFERRED_QUALITY',
    'UNIT_PRICE': 'AVG_UNIT_PRICE',
    'BOX_COUNT': 'TOTAL_BOXES_PURCHASED'
})

user_profiles.reset_index(inplace=True)

user_profiles.head()

,MAGIC_KEY,MOST_FREQUENT_BOX_ID,AVG_MILK,AVG_MEAT,PREFERRED_DELIVERY_OPTION,PREFERRED_QUALITY,AVG_UNIT_PRICE,TOTAL_BOXES_PURCHASED
0,249670911D8,231,4.25,1.2,Delivery from Collection Point,Standard,12.05,2
1,249751FC4DD,260,10.00,1.8,Delivery from Collection Point,Standard,17.98,1
2,24978027606,27,0.00,2.9,Home Delivery - CoD,Premium,15.96,1
3,24979164422,246,0.00,2.5,Delivery from Collection Point,Standard,13.96,1
4,2497B8B4FDA,255,5.00,2.7,Delivery from Collection Point,Standard,16.97,2


In [11]:
# Count unique BOX_IDs for each QUALITY category
box_quality_counts = df.groupby('QUALITY')['BOX_ID'].nunique().reset_index(name='NUM_BOXES')

# Count unique BOX_IDs for each DELIVERY_OPTION category
box_delivery_counts = df.groupby('DELIVERY_OPTION')['BOX_ID'].nunique().reset_index(name='NUM_BOXES')

# Display the results
print("Unique BOX_IDs per QUALITY category:")
print(box_quality_counts)
print("\nUnique BOX_IDs per DELIVERY_OPTION category:")
print(box_delivery_counts)


Unique BOX_IDs per QUALITY category:
    QUALITY  NUM_BOXES
0   Premium        197
1  Standard         93

Unique BOX_IDs per DELIVERY_OPTION category:
                   DELIVERY_OPTION  NUM_BOXES
0   Delivery from Collection Point        179
1              Home Delivery - CoD         75
2  Home Delivery - Digital Payment         36


In [12]:
# Group by MAGIC_KEY to calculate the number of unique BOX_IDs and total purchases
purchase_patterns = df.groupby('MAGIC_KEY').agg(
    unique_boxes=pd.NamedAgg(column='BOX_ID', aggfunc='nunique'),
    total_purchases=pd.NamedAgg(column='BOX_ID', aggfunc='count')
).reset_index()

# Add a new column to indicate if a MAGIC_KEY tends to buy the same box or not
purchase_patterns['same_box_preference'] = purchase_patterns['unique_boxes'] == 1

purchase_patterns.head()

,MAGIC_KEY,unique_boxes,total_purchases,same_box_preference
0,249670911D8,2,2,False
1,249751FC4DD,1,1,True
2,24978027606,1,1,True
3,24979164422,1,1,True
4,2497B8B4FDA,2,2,False


In [14]:
same_box_proportion = purchase_patterns['same_box_preference'].mean()

# Calculate the proportion of MAGIC_KEYs that buy different boxes
diff_box_proportion = 1 - same_box_proportion

# Count the number of MAGIC_KEYs for each preference
preference_counts = purchase_patterns['same_box_preference'].value_counts()

# Display the results
print(f"Proportion of customers always buying the same box: {same_box_proportion:.2f}")
print(f"Proportion of customers buying different boxes: {diff_box_proportion:.2f}")
print("\nCounts for each preference:")
print(preference_counts)

Proportion of customers always buying the same box: 0.67
Proportion of customers buying different boxes: 0.33

Counts for each preference:
same_box_preference
True     852999
False    421088
Name: count, dtype: int64


In [16]:
# First, filter out the MAGIC_KEYs that have bought more than one unique BOX_ID
multiple_box_keys = purchase_patterns[purchase_patterns['unique_boxes'] > 1]['MAGIC_KEY']

# Now create a DataFrame with the MAGIC_KEY and the corresponding unique BOX_ID if there's only one
# or 0 otherwise
purchase_patterns_2 = df.groupby('MAGIC_KEY').agg({
    'BOX_ID': lambda x: x.iloc[0] if x.nunique() == 1 else 0
}).reset_index()

# Now merge this with the purchase_patterns DataFrame
purchase_patterns_2 = purchase_patterns_2.merge(purchase_patterns, on='MAGIC_KEY', how='left')

# Rename the columns appropriately
purchase_patterns_2.rename(columns={'BOX_ID_x': 'same_box_id', 'BOX_ID_y': 'unique_boxes'}, inplace=True)


     MAGIC_KEY  BOX_ID  unique_boxes  total_purchases  same_box_preference
0  249670911D8       0             2                2                False
1  249751FC4DD     260             1                1                 True
2  24978027606      27             1                1                 True
3  24979164422     246             1                1                 True
4  2497B8B4FDA       0             2                2                False


In [17]:
purchase_patterns_2.head()

,MAGIC_KEY,BOX_ID,unique_boxes,total_purchases,same_box_preference
0,249670911D8,0,2,2,False
1,249751FC4DD,260,1,1,True
2,24978027606,27,1,1,True
3,24979164422,246,1,1,True
4,2497B8B4FDA,0,2,2,False


In [18]:
# Define a custom aggregation function to collect BOX_IDs into a list for each MAGIC_KEY
def collect_box_ids(series):
    return list(series.unique()) if len(series.unique()) > 1 else series.unique().tolist()

# Group by MAGIC_KEY and aggregate BOX_IDs
purchase_patterns_3 = df.groupby('MAGIC_KEY')['BOX_ID'].agg(collect_box_ids).reset_index()

# Now merge this with the original purchase_patterns DataFrame
purchase_patterns_3 = purchase_patterns_3.merge(purchase_patterns[['MAGIC_KEY', 'unique_boxes', 'total_purchases', 'same_box_preference']], on='MAGIC_KEY', how='left')

purchase_patterns_3.head()


,MAGIC_KEY,BOX_ID,unique_boxes,total_purchases,same_box_preference
0,249670911D8,"[231, 245]",2,2,False
1,249751FC4DD,[260],1,1,True
2,24978027606,[27],1,1,True
3,24979164422,[246],1,1,True
4,2497B8B4FDA,"[255, 260]",2,2,False


In [19]:
purchase_patterns_3['unique_boxes'].value_counts()

unique_boxes
1     852999
2     299452
3      93442
4      22794
5       4500
6        751
7        114
8         30
9          3
13         1
14         1
Name: count, dtype: int64

In [20]:
# Filter the DataFrame for rows where same_box_preference is True
same_box_preference_df = purchase_patterns_2[purchase_patterns_2['same_box_preference']]

# Now select only the MAGIC_KEY and BOX_ID columns
same_box_preference_df = same_box_preference_df[['MAGIC_KEY', 'BOX_ID']]

same_box_preference_df.head()

,MAGIC_KEY,BOX_ID
1,249751FC4DD,260
2,24978027606,27
3,24979164422,246
6,2498A873821,142
7,2498BF1EB04,143


In [22]:
same_box_preference_df['BOX_ID'].value_counts().sum()

852999

In [25]:
# Convert list of BOX_IDs to tuples to make them hashable and able to be set elements
purchase_patterns_3['BOX_ID_TUPLE'] = purchase_patterns_3['BOX_ID'].apply(tuple)

# Find unique combinations using a set
unique_combinations = set(purchase_patterns_3['BOX_ID_TUPLE'])

# Count of unique combinations
count_unique_combinations = len(unique_combinations)

# Display the unique combinations and their count
print(f"Total unique combinations of BOX_ID: {count_unique_combinations}")

Total unique combinations of BOX_ID: 24119


In [29]:
problem = pd.read_csv(r'C:\Users\Admin\Desktop\Datathon\Problem 2\problem_2.csv')
problem.shape

(43518, 1)

In [30]:
# Merge the datasets based on the common MAGIC_KEY
merged_df = problem.merge(same_box_preference_df, on='MAGIC_KEY', how='left')

# Fill NaN values in the 'BOX_ID' column with 0
merged_df['BOX_ID'] = merged_df['BOX_ID'].fillna(0).astype(int)

In [37]:
merged_df.sample(5)

,MAGIC_KEY,BOX_ID
5444,2BD263FD499,0
27176,2C49A8E2D31,0
31451,2C1F0D815C7,0
34926,2BE552CDFED,0
18053,2C27F4354E1,162


In [35]:
# Count the number of rows where BOX_ID is 0
zero_box_count = (merged_df['BOX_ID'] == 0).sum()

print("Number of boxes with BOX_ID equal to 0:", zero_box_count)

Number of boxes with BOX_ID equal to 0: 37455


In [38]:
purchase_patterns_3.sample(5)

,MAGIC_KEY,BOX_ID,unique_boxes,total_purchases,same_box_preference,BOX_ID_TUPLE,has_repeated_box
361934,2BC8FEC0BB7,"[76, 96]",2,2,False,"(76, 96)",False
377586,2BCD30061B3,"[8, 31, 137, 143]",4,8,False,"(8, 31, 137, 143)",False
310432,29D96A95777,"[201, 202]",2,2,False,"(201, 202)",False
897391,2C85143221B,"[23, 42]",2,2,False,"(23, 42)",False
769925,2C4DEEEA368,[119],1,1,True,"(119,)",False


In [39]:
# Group by MAGIC_KEY to calculate the total purchases and create a list of BOX_IDs with repetitions
user_profiles_2 = df.groupby('MAGIC_KEY').agg(
    total_purchases=pd.NamedAgg(column='BOX_COUNT', aggfunc='sum'),
    box_id_list=pd.NamedAgg(column='BOX_ID', aggfunc=list)
).reset_index()

,MAGIC_KEY,total_purchases,box_id_list
774053,2C513A2928A,1,[143]
712763,2C347335F4B,2,"[94, 101]"
955605,2C99959E7F3,1,[78]
811939,2C629959781,1,[12]
664289,2C242435201,1,[76]


In [42]:
user_profiles_2.sample(5)

,MAGIC_KEY,total_purchases,box_id_list
51093,28E0634ABBE,2,"[205, 205]"
987382,2CA3E3DCF9D,1,[143]
900015,2C86062FFC5,3,"[162, 163, 163]"
840643,2C6EE8025C8,3,"[94, 103, 106]"
576838,2C05C94C66F,1,[162]


In [44]:
# Extract year and month from PURCHASE_DATE into a new 'MONTH' column
df['MONTH'] = df['PURCHASE_DATE'].dt.to_period('M')

# Group by MAGIC_KEY and MONTH, and list BOX_IDs for each group
monthly_purchases = df.groupby(['MAGIC_KEY', 'MONTH'])['BOX_ID'].apply(list).unstack(fill_value=[])

# Reset the index to make MAGIC_KEY a column again
monthly_purchases.reset_index(inplace=True)

MONTH,MAGIC_KEY,2018-10,2018-11,2018-12,2019-01,2019-02
0,249670911D8,[],[245],[],[],[231]
1,249751FC4DD,[260],[],[],[],[]
2,24978027606,[27],[],[],[],[]
3,24979164422,[],[246],[],[],[]
4,2497B8B4FDA,[260],[255],[],[],[]


In [47]:
monthly_purchases.sample(5)

MONTH,MAGIC_KEY,2018-10,2018-11,2018-12,2019-01,2019-02
8136,28D262E5E0B,[],[],"[203, 203, 203]",[272],[]
501571,2BEDB6F96E8,[163],[163],[163],[162],[]
360516,2BC8A57DB0F,[],[],[],[],[78]
508731,2BEFF7868AB,[],[],[],[],[106]
124885,28F8C1C3652,[],[258],[258],[258],[258]


In [48]:
stacked_user_profile = pd.merge(monthly_purchases, user_profiles_2, on='MAGIC_KEY', how='outer')

In [54]:
stacked_user_profile.sample(10)

,MAGIC_KEY,2018-10,2018-11,2018-12,2019-01,2019-02,total_purchases,box_id_list
446906,2BDF324BBE9,[],[],[134],[77],[],2,"[77, 134]"
1010438,2CAAFF9E6B7,[89],[89],[89],"[89, 89]",[90],6,"[89, 89, 89, 89, 89, 90]"
288893,29756557499,[],[],[268],[221],[272],3,"[221, 268, 272]"
650591,2C1F16FAE35,[89],[],[],[134],[],2,"[89, 134]"
78202,28EAE58B2B5,[],[258],[],[258],[],2,"[258, 258]"
306298,29CDDCACB95,[],[245],[],[],[],1,[245]
818042,2C6567CD87B,[],[],[],[],[77],1,[77]
588990,2C09C54822A,[],[],[163],[],[],1,[163]
970266,2C9E08CD078,[],[],[],[],[108],1,[108]
363866,2BC994A0A11,[],[],[],[],[126],1,[126]


In [64]:
# Create a new DataFrame filtering out MAGIC_KEYs with total_purchases greater than 3
filtered_df = stacked_user_profile[stacked_user_profile['total_purchases'] > 4]

# Define a function to find the most frequent BOX_ID in the list
def most_frequent_box_id(box_id_list):
    if box_id_list:
        return max(set(box_id_list), key=box_id_list.count)
    return None  # Return None or a default value if the list is empty

# Apply the function to create a new column for the most frequent BOX_ID
filtered_df['most_frequent_box_id'] = filtered_df['box_id_list'].apply(most_frequent_box_id)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13212\1756369491.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['most_frequent_box_id'] = filtered_df['box_id_list'].apply(most_frequent_box_id)


In [65]:
filtered_df.head()

,MAGIC_KEY,2018-10,2018-11,2018-12,2019-01,2019-02,total_purchases,box_id_list,most_frequent_box_id
15,249C911B64A,"[247, 255]",[255],[255],"[255, 255]",[255],7,"[247, 255, 255, 255, 255, 255, 255]",255
25,249EA714089,"[258, 260]",[258],[],"[258, 258]",[258],6,"[258, 258, 258, 258, 258, 260]",258
27,249EDC9D2E5,[131],[163],[163],[144],[144],5,"[131, 144, 144, 163, 163]",144
30,249EDFFF231,[246],[246],[246],[245],[245],5,"[245, 245, 246, 246, 246]",246
32,249EF27B70C,[143],[143],[143],[142],[142],5,"[142, 142, 143, 143, 143]",143


In [66]:
# Define a function to find the most frequent BOX_ID and its frequency in the list
def most_frequent_box_id_and_frequency(box_id_list):
    if box_id_list:
        # Finding the most frequent box ID
        most_frequent = max(set(box_id_list), key=box_id_list.count)
        # Counting how often it appears in the list
        frequency = box_id_list.count(most_frequent)
        return most_frequent, frequency
    return None, 0  # Return None or a default value for ID and 0 for frequency if the list is empty

# Assume 'filtered_df' is your DataFrame and it already contains 'most_frequent_box_id' correctly
# Now apply the function to get the frequency
filtered_df['frequency'] = filtered_df['box_id_list'].apply(
    lambda x: x.count(max(set(x), key=x.count)) if x else 0
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13212\2900635744.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['frequency'] = filtered_df['box_id_list'].apply(


In [69]:
filtered_df.sample(10)

,MAGIC_KEY,2018-10,2018-11,2018-12,2019-01,2019-02,total_purchases,box_id_list,most_frequent_box_id,frequency
861096,2C76D7F4221,[143],[143],[143],[144],[144],5,"[143, 143, 143, 144, 144]",143,3
705474,2C31BEC77A0,"[89, 172, 174]",[174],"[174, 174]","[174, 174]","[174, 174]",10,"[89, 172, 174, 174, 174, 174, 174, 174, 174, 174]",174,8
913835,2C8AFEC502C,[107],[143],"[103, 143]",[174],[174],6,"[103, 107, 143, 143, 174, 174]",143,2
305246,29CB20D10F7,"[201, 216]","[215, 215]",[],[217],"[204, 217]",7,"[201, 204, 215, 215, 216, 217, 217]",215,2
828571,2C69C62D4CB,[143],[143],[174],[174],[174],5,"[143, 143, 174, 174, 174]",174,3
1140890,2CD5A060B99,"[31, 51]",[166],[],[44],[174],5,"[31, 44, 51, 166, 174]",166,1
837006,2C6D5BC2F9B,[143],[143],"[143, 143]",[144],[],5,"[143, 143, 143, 143, 144]",143,4
634907,2C19A9B1ABB,[106],[106],[103],[174],[174],5,"[103, 106, 106, 174, 174]",106,2
263307,295D92E653A,[205],[206],[206],[205],[205],5,"[205, 205, 205, 206, 206]",205,3
127149,28F962C45A8,[231],[231],[231],[231],[231],5,"[231, 231, 231, 231, 231]",231,5


In [70]:
# First Dataset: MAGIC_KEYs with frequency >= 5
dataset1 = filtered_df[filtered_df['frequency'] >= 5][['MAGIC_KEY', 'most_frequent_box_id']]
dataset1.columns = ['MAGIC_KEY', 'BOX_ID']

In [71]:
dataset1.head()

,MAGIC_KEY,BOX_ID
15,249C911B64A,255
25,249EA714089,258
68,24A5BD7BB7D,258
143,24B282735A4,255
211,24C2128921E,238


In [73]:
# Merge merged_df with dataset1 on 'MAGIC_KEY' to get the BOX_IDs from dataset1
merged_with_update = pd.merge(merged_df, dataset1, on='MAGIC_KEY', how='left', suffixes=('', '_new'))

# Update the BOX_ID in merged_df only where it is 0 and a non-null update value is available
merged_with_update['BOX_ID'] = merged_with_update.apply(
    lambda row: row['BOX_ID_new'] if row['BOX_ID'] == 0 and pd.notna(row['BOX_ID_new']) else row['BOX_ID'],
    axis=1
)

# Drop the temporary new BOX_ID column as it's no longer needed
merged_with_update.drop(columns='BOX_ID_new', inplace=True)

In [74]:
merged_with_update.sample(5)

,MAGIC_KEY,BOX_ID
40380,2BE5FC725A4,0.0
230,2C175915DDC,0.0
5276,2BD175D6F59,0.0
18065,2C28018C428,0.0
26113,2C22848644F,0.0


In [79]:
# Assuming merged_with_update is the updated DataFrame from previous steps

# Get the unique values from the 'BOX_ID' column
unique_box_ids = merged_with_update['BOX_ID'].unique()

# Print the unique values
print("Unique BOX_ID values in the updated DataFrame:")
print(unique_box_ids)


Unique BOX_ID values in the updated DataFrame:
[  0. 144. 258. 255. 238. 271. 201.  27. 119.  89.   8. 131.  96.  65.
 174. 142. 143.  77. 106. 118.  94. 162. 203. 213. 272. 246. 245. 237.
 231. 278. 233. 248. 198. 215. 276. 227. 204. 220. 228. 270. 221. 268.
 252. 103. 166. 137. 130. 108. 134.  80.  44. 101.  99.  51.  78.  13.
 133. 182.   4.  56. 104. 177.  69. 205.   3. 126. 254. 269.  41. 217.]


In [80]:
# Count the number of rows where BOX_ID is 0
zero_box_count = (merged_with_update['BOX_ID'] == 0).sum()

print("Number of boxes with BOX_ID equal to 0:", zero_box_count)

Number of boxes with BOX_ID equal to 0: 36143


In [81]:
# Filter for February 2019
feb_2019_data = df[df['PURCHASE_DATE'].dt.month == 2][df['PURCHASE_DATE'].dt.year == 2019]

# Group by MAGIC_KEY and BOX_ID, count occurrences
grouped = feb_2019_data.groupby(['MAGIC_KEY', 'BOX_ID']).size().reset_index(name='Count')

# Find the BOX_ID with the highest count for each MAGIC_KEY
# First, sort the data by Count in descending order to prepare for picking the top most
grouped_sorted = grouped.sort_values(by=['MAGIC_KEY', 'Count'], ascending=[True, False])

# Drop duplicates to keep the row with the highest 'Count' per MAGIC_KEY
most_purchased = grouped_sorted.drop_duplicates(subset='MAGIC_KEY')

# Rename the columns appropriately for output
most_purchased.columns = ['MAGIC_KEY', 'Most_Purchased_BOX_ID_Feb19', 'Frequency_Feb19']

most_purchased.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_13212\2803145736.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  feb_2019_data = df[df['PURCHASE_DATE'].dt.month == 2][df['PURCHASE_DATE'].dt.year == 2019]


,MAGIC_KEY,Most_Purchased_BOX_ID_Feb19,Frequency_Feb19
0,249670911D8,231,1
1,2497E36D1BF,31,2
2,2498CA210F2,246,1
3,2498CF2157B,254,1
4,2499B9E0C1C,89,2


In [82]:
# Merge merged_with_update with most_purchased on 'MAGIC_KEY' to get the Most_Purchased_BOX_ID_Feb19
merged_with_box_ids = pd.merge(merged_with_update, most_purchased[['MAGIC_KEY', 'Most_Purchased_BOX_ID_Feb19']], on='MAGIC_KEY', how='left')

# Update the BOX_ID in merged_with_update where it is 0.0 with Most_Purchased_BOX_ID_Feb19
merged_with_box_ids['BOX_ID'] = merged_with_box_ids.apply(
    lambda row: row['Most_Purchased_BOX_ID_Feb19'] if row['BOX_ID'] == 0.0 else row['BOX_ID'],
    axis=1
)

# Drop the temporary Most_Purchased_BOX_ID_Feb19 column as it's no longer needed
merged_with_box_ids.drop(columns='Most_Purchased_BOX_ID_Feb19', inplace=True)

In [83]:
merged_with_box_ids.sample(5)

,MAGIC_KEY,BOX_ID
17672,2C24FEA0335,122.0
37005,2C246236F0B,134.0
32490,2C42CEB9171,142.0
4588,2BCD9F25FD8,27.0
31648,2C25F2B13E5,90.0


In [84]:
# Get the unique values from the 'BOX_ID' column
unique_box_ids = merged_with_box_ids['BOX_ID'].unique()

# Print the unique values
print("Unique BOX_ID values in the updated DataFrame:")
print(unique_box_ids)

Unique BOX_ID values in the updated DataFrame:
[141. 144. 230. 270. 162. 108.  31. 258. 217.  nan 255. 238. 220. 204.
 272. 271. 237. 201. 119. 110.  94.  27.  42. 131.  89. 138.   8. 126.
   9.  95. 118. 174.  96.  65.  23.  90. 142. 124. 143. 134. 103. 106.
  76.  28.  77.  44. 152. 123. 246. 245. 203. 213. 231. 254. 228. 248.
 200. 251. 221. 269. 252. 209. 276. 278. 233. 205. 198. 222. 215. 227.
 268. 277. 206. 223. 166.  78. 155.  72. 137. 130.  45.  80. 149.  56.
 101.  99.  51.  29.  24. 154. 148.  26. 122.   6.   3. 104. 158.  69.
   4.  68.  13.  32. 133. 177. 178.  55. 182. 196.  59.  12.  46. 185.
 194.  52.  67. 147.  41. 187. 121. 165.  43.]


In [85]:
merged_with_box_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43518 entries, 0 to 43517
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MAGIC_KEY  43518 non-null  object 
 1   BOX_ID     36613 non-null  float64
dtypes: float64(1), object(1)
memory usage: 680.1+ KB


In [87]:
# Replace NaNs with 0 (or another placeholder integer if 0 is meaningful in your context)
merged_with_box_ids['BOX_ID'].fillna(0, inplace=True)

# Convert BOX_ID to integer
merged_with_box_ids['BOX_ID'] = merged_with_box_ids['BOX_ID'].astype('int32')

In [88]:
# Count the number of rows where BOX_ID is 0
zero_box_count = (merged_with_box_ids['BOX_ID'] == 0).sum()

print("Number of boxes with BOX_ID equal to 0:", zero_box_count)

Number of boxes with BOX_ID equal to 0: 6905


In [89]:
merged_with_box_ids.sample(5)

,MAGIC_KEY,BOX_ID
11448,2BF34C33724,142
21943,29D916BCB5F,220
4152,2BCB857BD56,0
29749,2BEAD8055B2,8
2449,2A5B302651A,144


In [92]:
# Merge merged_with_box_ids with user_profiles on 'MAGIC_KEY' to get the MOST_FREQUENT_BOX_ID
merged_with_most_frequent = pd.merge(merged_with_box_ids, user_profiles[['MAGIC_KEY', 'MOST_FREQUENT_BOX_ID']], on='MAGIC_KEY', how='left')

# Conditionally update BOX_ID where it is 0 using MOST_FREQUENT_BOX_ID
merged_with_most_frequent['BOX_ID'] = merged_with_most_frequent.apply(
    lambda row: row['MOST_FREQUENT_BOX_ID'] if row['BOX_ID'] == 0 else row['BOX_ID'],
    axis=1
)

# Drop the temporary MOST_FREQUENT_BOX_ID column as it's no longer needed
merged_with_most_frequent.drop(columns='MOST_FREQUENT_BOX_ID', inplace=True)

# Display the updated DataFrame to verify the changes
merged_with_most_frequent.sample(5)


,MAGIC_KEY,BOX_ID
23320,2BD60E2054B,162
24556,2BEFFAFA70B,144
4313,2BCC56EBEC4,42
12390,2BF9D70263D,162
34220,2BD73ACF839,142


In [93]:
# Count the number of rows where BOX_ID is 0
zero_box_count = (merged_with_most_frequent['BOX_ID'] == 0).sum()

print("Number of boxes with BOX_ID equal to 0:", zero_box_count)

Number of boxes with BOX_ID equal to 0: 0


In [94]:
#merged_with_most_frequent.to_csv('submission.csv', index=False)